In [1]:
from pts.model.tempflow import TempFlowEstimator
from pts.model.time_grad import TimeGradEstimator
from pts.model.transformer_tempflow import TransformerTempFlowEstimator
from pts import Trainer
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

/Users/abhishek.sawalkar/opt/anaconda3/envs/ml3/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/abhishek.sawalkar/opt/anaconda3/envs/ml3/lib/python3.6/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import torch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [28]:
df=pd.read_csv("../dataset/AirQualityUCI_Italy.csv",sep=";",parse_dates={ 'DateTime': ['Date','Time']},index_col='DateTime',usecols=[0,1,3])
df.dropna(inplace=True)
df.index=pd.to_datetime(df.index, format='%d/%m/%Y %H.%M.%S')
# df.rename(columns = {'PT08.S1(CO)':np.int64(0)}, inplace = True)


In [19]:
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(df, test_size=0.2)
train_size=(len(df)*8)//10
print(train_size)
train, test = df.iloc[:train_size],df.iloc[train_size:]


7485


In [20]:
#function for describing dataframe
def desc_df(df):
    print("\nType:",type(df))
    print("\nShape:",df.shape)
    print("\nDatatypes:",df.dtypes)
    print("\nDescription:",df.describe())
    print("\nHead:",df.head(10))

In [21]:
desc_df(train)


Type: <class 'pandas.core.frame.DataFrame'>

Shape: (7485, 1)

Datatypes: PT08.S1(CO)    float64
dtype: object

Description:        PT08.S1(CO)
count  7485.000000
mean   1052.568604
std     327.724860
min    -200.000000
25%     922.000000
50%    1055.000000
75%    1228.000000
max    2040.000000

Head:                      PT08.S1(CO)
DateTime                        
2004-03-10 18:00:00       1360.0
2004-03-10 19:00:00       1292.0
2004-03-10 20:00:00       1402.0
2004-03-10 21:00:00       1376.0
2004-03-10 22:00:00       1272.0
2004-03-10 23:00:00       1197.0
2004-03-11 00:00:00       1185.0
2004-03-11 01:00:00       1136.0
2004-03-11 02:00:00       1094.0
2004-03-11 03:00:00       1010.0


In [31]:
df["PT08.S1(CO)"][:7485].shape

(7485,)

In [29]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName
train_data=ListDataset(
    [
        {"start":df.index[0],"target":df["PT08.S1(CO)"][:7485]}
        # {"start":df.index[0],"target":train}
        # {"start":df.index[0],"target":df.iloc[0:7485,1:3],}
        # {
        #     FieldName.START:train.index[0],
        #     FieldName.TARGET:train
        # }
    ],
    freq = "1H"
    # ,one_dim_target=False
)

In [30]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
grouper_train = MultivariateGrouper()

train_data = grouper_train(train_data)
# train_data=list(train_data)*100



In [32]:
type(train_data)

gluonts.dataset.common.ListDataset

In [34]:
# def run():
#     for i in range(1500,10000):
#         try:

estimator = TimeGradEstimator(
    target_dim=[1],
    # target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=1872,
    context_length=1872,
    cell_type='GRU',
    input_size=28,
    freq="1H",
    loss_type='l2',
    scaling=True,
    diff_steps=100,
    beta_end=0.1,
    beta_schedule="linear",
    trainer=Trainer(device=device,
                    epochs=20,
                    learning_rate=1e-3,
                    num_batches_per_epoch=100,
                    batch_size=2,)
)
predictor = estimator.train(train_data)
#         except RuntimeError:
#             continue
#             # print(i)
#         except Exception as e:
#             print(i,e)
#             break
# run()


ValidationError: 1 validation error for TimeGradTrainingNetworkModel
target_dim
  value is not a valid integer (type=type_error.integer)